In [3]:
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utilities import import_data_files
import torch
import torch.nn as nn
import torch.optim as optim
import awkward as ak

In [29]:
class KernelSVM(nn.Module):
    def __init__(self, input_size=9801, output_size=1):
        super(KernelSVM, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x1, x2):
        x1 = x1.view(-1)
        x2 = x2.view(-1)
        
        kernel = torch.ger(x1, x2) ** 2
        
        kernel_flat = kernel.view(-1)
        return self.linear(kernel_flat)

In [6]:
def split_data(data, labels, train_ratio=0.8):
    """
    Splits the data and labels into training and testing sets.

    Args:
        data (torch.Tensor): The input data tensor.
        labels (torch.Tensor): The corresponding labels tensor.
        train_ratio (float, optional): The ratio of data to use for training. Defaults to 0.8.

    Returns:
        tuple: A tuple of four tensors: (train_data, train_labels, test_data, test_labels)
    """

    num_train_samples = int(train_ratio * len(data))

    train_data, test_data = torch.utils.data.random_split(data, [num_train_samples, len(data) - num_train_samples])
    train_labels, test_labels = torch.utils.data.random_split(labels, [num_train_samples, len(labels) - num_train_samples])
    print("Data type of training data:", train_data.dataset.dtype)
    print("Data type of training labels:", train_labels.dataset.dtype)
    print("Data type of testing data:", test_data.dataset.dtype)
    print("Data type of testing labels:", test_labels.dataset.dtype)
    return train_data, train_labels, test_data, test_labels

In [7]:
def hinge_loss(output, target):
    return torch.clamp(1 - output * target.view_as(output), min=0).mean()

In [8]:
def prepare_data(DFs):
    print("Data loaded from files")
    print("Converting data to numpy array...")
    
    accepted_numpy = ak.to_numpy(DFs[0]['SuperCell_ET'])
    rejected_numpy = ak.to_numpy(DFs[1]['SuperCell_ET'])
    
    print("Converting data to torch tensors...")
    accepted_tensor = torch.tensor(accepted_numpy, dtype=torch.float32)
    rejected_tensor = torch.tensor(rejected_numpy, dtype=torch.float32)
    
    print("Generating labels...")
    accepted_labels = torch.zeros(len(accepted_tensor), dtype=torch.float32)
    rejected_labels = torch.ones(len(rejected_tensor), dtype=torch.float32)
    
    print("Concatenating datasets...")
    data = torch.cat([accepted_tensor, rejected_tensor], dim=0)
    labels = torch.cat([accepted_labels, rejected_labels], dim=0)
    
    print("Shuffling data...")
    indices = torch.randperm(len(data))
    data = data[indices]
    labels = labels[indices]
    
    print("Splitting data...")
    dataset = torch.utils.data.TensorDataset(data, labels)
    train_size = int(0.8 * len(dataset))
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])
    
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)
    print("DONE")
    
    return train_loader, test_loader

In [25]:
def train_model(train_loader):
    # Hyperparameters
    input_size = 9801
    output_size = 1
    learning_rate = 0.01
    num_epochs = 10
    
    model = KernelSVM(output_size=output_size)
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0
        for x, y in train_loader:
            optimizer.zero_grad()
            
            output = model(x, x)
            loss = hinge_loss(output, y)
            
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(train_loader):.4f}')
    
    torch.save(model.state_dict(), 'svm_model.pth')
    return model

In [11]:
DFs = import_data_files(["l1calo_hist_ZMUMU_extended.root", "l1calo_hist_EGZ_extended.root"])

In [12]:
train_loader, test_loader = prepare_data(DFs)

Data loaded from files
Converting data to numpy array...
Converting data to torch tensors...
Generating labels...
Concatenating datasets...
Shuffling data...
Splitting data...


In [28]:
model = train_model(train_loader)

Epoch [1/10], Loss: 0.6140
Epoch [2/10], Loss: 0.6140
Epoch [3/10], Loss: 0.6140
Epoch [4/10], Loss: 0.6140
Epoch [5/10], Loss: 0.6140
Epoch [6/10], Loss: 0.6140
Epoch [7/10], Loss: 0.6140
Epoch [8/10], Loss: 0.6140
Epoch [9/10], Loss: 0.6140
Epoch [10/10], Loss: 0.6140
